# Gradient and Hessian using forward difference method

In this notebook, we will demonstrate how to numerically compute the gradient and Hessian of a function in **Julia** using forward difference method.

The *forward difference method* approximates the derivative definitions using a small step $h$:

$$
f'(x) \approx \frac{f(x + h) - f(x)}{h}
$$

For a function $f: \mathbb{R^n} \rightarrow \mathbb{R}$ where $X = [x_1, x_2, \cdots, x_n]^T$ the partial derivative of $f(X)$ with respect to $x_i$ is defined using forward difference method as follows:

$$
\frac{\partial f(X)}{\partial x_i} \approx \frac{f(X + he_i) - f(X)}{h}
$$

Where $e_i$ represents an n-dimensional vector consisting of all zeros, except for the $i$ th component, which is equal to 1.

Let's define $f(X) = x_1 \cos{(x_2x_3)} + x_2e^{x_3}$ where the partial derivatives are showed below:

$$
\begin{align*}
\frac{\partial f}{\partial x_1} &= \cos{(x_2x_3)} \\
\frac{\partial f}{\partial x_2} &= -x_1x_3 \sin{(x_2x_3)} + e^{x_3} \\
\frac{\partial f}{\partial x_3} &= -x_1x_2 \sin{(x_2x_3)} + x_2e^{x_3} \\
\end{align*}
$$

If we evalute at $X = [1, \pi/2, \pi/2]$:

$$
\begin{align*}
\frac{\partial f}{\partial x_1} &\approx -0.781211\\
\frac{\partial f}{\partial x_2} &\approx 3.82988 \\
\frac{\partial f}{\partial x_3} &\approx 6.57568  \\
\end{align*}
$$

To start, let's create the function to be derivated and a forward difference method:

In [1]:
f(X) = X[1]cos(X[2]X[3]) + X[2]ℯ^(X[3])

f (generic function with 1 method)

In [2]:
function forward_diff(f, X, i, h=sqrt(eps(Float64)))

    # Define the e_i vector
    ei = zeros(length(X))
    ei[i] = 1

    # Compute the partial derivative with respect to x_i
    ∂f = (f(X + h*ei) - f(X)) / h

    return ∂f
end

forward_diff (generic function with 2 methods)

Now let's compute the derivatives:

In [3]:
X = [1, π/2, π/2]

forward_diff(f, X, 1), forward_diff(f, X, 2), forward_diff(f, X, 3)

(-0.7812118530273438, 3.8298827409744263, 6.575685679912567)

We obtained identical values to those obtained using the analytical expressions:

In [4]:
cos(π^2/4), (-1*π/2)sin(π^2/4) + ℯ^(π/2), (-1*π/2)sin(π^2/4) + (π/2)ℯ^(π/2)

(-0.7812118921104881, 3.829882715615795, 6.575685534800751)

In fact, given that the gradient of a functions $f(X)$ is defined as:

$$
\nabla f(X) = \left[\frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, \cdots, \frac{\partial f}{\partial x_n}\right]^T
$$

We can compute the gradient, with **gradient_forward()** function that uses *forward difference method* to compute the derivatives:

In [5]:
function gradient_forward(f, X; h=sqrt(eps(Float64)))

    # Define the gradient with all zeros
    ∇f = zeros(length(X))

    for i = 1:length(X)
        ∇f[i] = forward_diff(f, X, i, h)
    end

    return ∇f
end

gradient_forward (generic function with 1 method)

In [6]:
α = gradient_forward(f, X)

α

3-element Vector{Float64}:
 -0.7812118530273438
  3.8298827409744263
  6.575685679912567

The *Hessian* of a multivariate function is a matrix containing all of the second derivatives with respect to the input. The second derivatives capture information about the local curvature of the function.

$$
\nabla^2 f(x) = 
\begin{pmatrix}
\frac{\partial^2 f(x)}{\partial x_1 \partial x_1} & \frac{\partial^2 f(x)}{\partial x_1 \partial x_2} & \cdots & \frac{\partial^2 f(x)}{\partial x_1 \partial x_n} \\
\vdots & \vdots &  & \vdots \\
\frac{\partial^2 f(x)}{\partial x_n \partial x_1} & \frac{\partial^2 f(x)}{\partial x_n \partial x_2} & \cdots & \frac{\partial^2 f(x)}{\partial x_n \partial x_n} \\
\end{pmatrix}
$$

In order to approximate the Hessian Matrix, we can approximate $H_{ij} = \frac{\partial^2 f}{\partial x_ix_j}$ using forward difference. If we define $u_j = \frac{\partial f}{\partial x_j}$ then we can approximate $\frac{\partial u_j}{\partial x_i}$:

$$
\frac{\partial u_j}{\partial x_i} = \frac{u_j(X + he_i) - u_j(X)}{h}
$$

Approximating $u_j$ with the forward difference method again

$$
u_j = \frac{\partial f}{\partial x_j} = \frac{f(X + he_j) - f(X)}{h}
$$

Then, evaluating at $X + he_i$ and $X$:

$$
u_j(X + he_i) = \frac{f(X + he_i + he_j) - f(X + he_i)}{h}
$$
$$
u_j(X) = \frac{f(X  + he_j) - f(X)}{h}
$$

Using these values we obtain:
$$
\frac{\partial u_j}{\partial x_i} = \frac{\frac{f(X + he_i + he_j) - f(X + he_i)}{h} - \frac{f(X  + he_j) - f(X)}{h}}{h}
$$

Given that $u_j = \frac{\partial f}{\partial x_j}$, then $\frac{\partial u_j}{\partial x_i} = \frac{\partial^2 f}{\partial x_i \partial x_j}$. Hence, the final expression is presented:

$$
    H_{ij} = \frac{\partial^2 f}{\partial x_i \partial x_j} = \frac{f(X + he_i + he_j) - f(X + he_i) - f(X + he_j) + f(X)}{h^2}
$$

Now, we are going to create a function to compute the Hessian given a point $X$ using the forward difference method, first we need a function to compute the partial derivatives:

In [7]:
# This function takes a vector X where the partial derivative is computed.
# i and j are subindices to compute ∂/∂x_i (∂f/∂x_j)
function partial_forward(f, X, i, j; h=1e-5)

    # Create e_i and e_j
    ei = zeros(length(X))
    ej = zeros(length(X))

    # Add non-zero component
    ei[i] = 1
    ej[j] = 1

    # Compute ∂^2f / ∂x_i ∂x_j
    ∂f2 =(f(X +  h*ei + h*ej) - f(X + h*ei) - f(X + h*ej) + f(X)) / (h^2)

    return ∂f2
end

partial_forward (generic function with 1 method)

In [8]:
# Compute the Hessian of f at given X, ∇^2f(X)
function hessian_forward(F, X, h=1e-5)

    H = [partial_forward(f, X, i, j, h=h) for i=1:length(X), j=1:length(X)]

    return H
end

hessian_forward (generic function with 2 methods)

The terms of the Hessian matrix for $f(X) = x_1\cos{(x_2x_3)} + x_2e^{x_3}$ are displayed below:

$$
H_{1,1} = 0 \, , \, H_{1,2} = -x_3\sin{(x_2x_3)} \, , \, H_{1,3} = -x_2\sin{(x_2x_3)}
$$
$$
H_{2, 1} = -x_3 \sin{(x_2x_3)} \, , \, H_{2,2} = -x_1x_3^2\cos{(x_2x_3)} \, , \, H_{2,3} = -x_1[x_2x_3\cos{(x_2x_3)} + \sin{(x_2x_3)}] + e^{x_3}
$$
$$
H_{3, 1} = -x_2\sin{(x_2x_3)} \, , \, H_{3, 2} = -x_1[x_2x_3\cos{(x_2x_3)} + \sin{(x_2x_3)}] + e^{x_3} \, , \, H_{3,3} = -x_1x_2^2\cos{(x_2x_3)} + x_2e^{x_3}
$$

Evaluated at the point $X = [1, \frac{\pi}{2}, \frac{\pi}{2}]$ the hessian is:


$$
\nabla^2 f(x) = 
\begin{pmatrix}
0 & -0.98059 & -0.98059 \\
-0.98059 & 1.92756 & 6.11377 \\
-0.98059  & 6.11377 & 9.48384
\end{pmatrix}
$$

Then we can compute the Hessian of any n-dimensional funcion given a point $X = [x_1, x_2, \cdots, x_n]^T$

In [18]:
Hessian_forward = hessian_forward(f, X, 1e-4)

3×3 Matrix{Float64}:
  0.0       -0.980498  -0.980498
 -0.980498   1.92781    6.1145
 -0.980498   6.1145     9.48484

These values are identical that if we compute then using the analytical functions:

Each term of the Hessian for $f(X) = x_1\cos{(x_2x_3)} + x_2e^{x_3}$ is showed:

$$
H_{1,1} = 0 \, , \, H_{1,2} = -x_3\sin{(x_2x_3)} \, , \, H_{1,3} = -x_2\sin{(x_2x_3)}
$$
$$
H_{2, 1} = -x_3 \sin{(x_2x_3)} \, , \, H_{2,2} = -x_1x_3^2\cos{(x_2x_3)} \, , \, H_{2,3} = -x_1[x_2x_3\cos{(x_2x_3)} + \sin{(x_2x_3)}] + e^{x_3}
$$
$$
H_{3, 1} = -x_2\sin{(x_2x_3)} \, , \, H_{3, 2} = -x_1[x_2x_3\cos{(x_2x_3)} + \sin{(x_2x_3)}] + e^{x_3} \, , \, H_{3,3} = -x_1x_2^2\cos{(x_2x_3)} + x_2e^{x_3}
$$

Evaluated at the point $X = [1, \frac{\pi}{2}, \frac{\pi}{2}]$ the hessian is:

In [11]:
h11 = 0
h12 = -X[3]sin(X[2]X[3])
h13 = -X[2]sin(X[2]X[3])

h21 = -X[3]sin(X[2]X[3])
h22 = -X[1]*(X[3]^2)cos(X[2]X[3])
h23 = -X[1]*(X[2]X[3]cos(X[2]X[3]) + sin(X[2]X[3])) + ℯ^X[3]

h31 = -X[2]sin(X[2]X[3])
h32 = -X[1]*(X[2]X[3]cos(X[2]X[3]) + sin(X[2]X[3])) + ℯ^(X[3])
h33 = -X[1]*(X[2]^2)cos(X[2]X[3]) + X[2]ℯ^X[3]

Hessian = [h11 h12 h13; h21 h22 h23; h31 h32 h33]

3×3 Matrix{Float64}:
  0.0       -0.980595  -0.980595
 -0.980595   1.92756    6.11377
 -0.980595   6.11377    9.48384

To the eigenvalues of the Hessian matrix $\lambda_1, \lambda_2, \cdots, \lambda_n$, we can utilize the **eigvals()** function from `LinearAlgebra.jl` package. These eigenvalues are highly valuable in optimization tasks.

In [19]:
using LinearAlgebra

eigvals(Hessian_forward)

3-element Vector{Float64}:
 -1.5799902691005632
 -0.03811980869608502
 13.030755928380307